# **FOOD COLLECTOR**

### **DESCRIPCIÓN**

El reto consiste en desarrollar un sistema multiagente para resolver una tarea cooperativa en un entorno 20x20 dinámicamente cambiante. El entorno del sistema multiagente es un mundo similar a una cuadrícula donde los agentes pueden moverse de su posición a una celda vecina si ya no hay ningún agente en esa ranura. En este entorno, la comida puede aparecer en cualquier celda menos en una. La celda especial, en la que no puede aparecer comida, se considera un depósito donde los agentes pueden traer y almacenar su comida. Un agente puede sólo puede saber si hay comida en una celda, si está visitándola. Inicialmente, la comida se coloca en algunas celdas aleatorias. Durante la ejecución, puede aparecer comida adicional dinámicamente en celdas seleccionadas al azar, excepto en la celda del depósito. Los agentes pueden tener/desempeñar diferentes roles (como explorador o recolector), comunicarse y cooperar para encontrar y recolectar alimentos de manera eficiente y efectiva.

#### **PUNTOS A CONSIDERAR**

- Inicialmente no hay comida en el entorno.
- La semilla para generación de números aleatorios será 12345.
- El depósito será generado al azar.
- Cada 5 segundos se colocará una unidad de comida en algunas celdas.
- La cantidad de celdas en las que colocará una unidad comida será definida al azar (entre 2 y 5 celdas).
- Se colocará un total de 47 unidades de comida.
- Número total de pasos (steps): 1500.
- La cantidad total de alimentos que se puede almacenar en el depósito es infinito.
- Hay un total de 5 agentes.
- Cuando una unidad de comida es encontrado por un explorador o por un agente que ya lleva la comida, la posición de la comida se marca y se comunica a otros agentes.
- Cuando un recolector encuentra una unidad comida, lo carga (gráficamente deberá cambia su forma para indicar que lleva comida). La capacidad máxima de comida que puede llevar un agentes es UNA unidad de comida.
- Inicialmente, los agentes no son informados sobre la posición del depósito, pero una vez que lo encuentran, todos saben dónde está.

In [ ]:
from mesa import Agent, Model

from mesa.space import SingleGrid

from mesa.time import RandomActivation

from mesa.datacollection import DataCollector

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

import numpy as np
import pandas as pd
import math

import seaborn as sns

In [ ]:
# Agent Class
class ExplorerAgent(Agent):
    def __init__(self, id, model):
        super().__init__(id, model)
        self.random.seed(12345)
        self.type = 1

    def step(self):
        if not self.model.hasStorage:
            self.check_storage()
        else:
            self.lookforfood()

    def move(self):
        neighbors = self.model.grid.get_neighborhood(self.pos, 
                                                     moore = True, 
                                                     include_center = False
        )

        is_possible = [step for step in neighbors if self.model.grid.is_cell_empty(step)]

        if is_possible:
            new_position = self.random.choice(is_possible)
            self.model.grid.move_agent(self, new_position)

    def check_storage(self):
        x,y = self.pos
        if self.model.floor[x][y] == 10:
            self.model.hasStorage = True
            self.model.position_storage = (x, y)
        self.move()
    
    def lookforfood(self):
        x,y = self.pos
        if self.model.floor[x][y] > 0 and self.model.floor[x][y] < 10:
            if self.pos not in self.model.positions_food:
                self.model.positions_food.append(self.pos)
        self.move()

In [ ]:
class CollectorAgent(Agent):
    def __init__(self, id, model):
        super().__init__(id, model)
        self.random.seed(12345)
        self.hasFood = False
        self.target = None
        self.type = 2

    def shortest_distance(self, target_position):
        x1, y1 = self.pos
        x2, y2 = target_position

        # Manhattan distance
        return abs(x2 - x1) + abs(y2 - y1)
    
    def getTarget(self):
        distances = [self.shortest_distance(food) for food in self.model.positions_food]
        if distances:
            closests_food_index = distances.index(min(distances))
            closest_food_pos = self.model.positions_food[closests_food_index]
            self.target = closest_food_pos

    def random_move(self):
        neighbors = self.model.grid.get_neighborhood(self.pos, 
                                                     moore = True, 
                                                     include_center = False
        )

        is_possible = [step for step in neighbors if self.model.grid.is_cell_empty(step)]

        if is_possible:
            new_position = self.random.choice(is_possible)
            self.model.grid.move_agent(self, new_position)

    def move(self):
        x, y = self.pos
        x2, y2 = self.target

        dirx = x2 - x
        diry = y2 - y

        a = x
        b = y
        
        if dirx > 0:
            a = a + 1 
        elif dirx < 0:
            a = a - 1

        if diry > 0:
            b = b + 1
        elif diry < 0:
            b = b - 1

        new_position = (a, b)
                
        if self.model.grid.is_cell_empty(new_position):
            self.model.grid.move_agent(self, new_position)
        else:
            self.random_move()

    def pickup(self):
        x, y = self.pos
        if self.model.floor[x][y] > 0 and self.model.floor[x][y] < 10:
            if (x, y) in self.model.positions_food:
                self.hasFood = True
                self.target = self.model.position_storage
                self.model.floor[x][y] -= 1
                if (x, y) in self.model.positions_food and self.model.floor[x][y] == 0:
                    self.model.positions_food.remove((x, y))

    def drop(self):
        self.hasFood = False
        self.target = None
        self.model.collected_food += 1

    def step(self): 
        if self.pos == self.target:
            self.target = None

        if self.hasFood:
            if self.model.hasStorage:
                if self.pos == self.model.position_storage:
                    self.drop()
                else:
                    self.move()
        else:
            self.pickup()
            if self.hasFood:
                return
            else:
                if self.target != None:
                    self.move()
                else:
                    self.getTarget()

In [ ]:
# Model Class
class FoodModel(Model):
    def __init__(self, width, height, num_explorers, num_collectors, count_food):
        self.random.seed(12345)
        self.hasStorage = False
        self.num_explorers = num_explorers
        self.num_collectors = num_collectors
        self.count_food = count_food
        self.positions_food = []
        self.position_storage = None
        self.collected_food = 0
        self.placed_food = 0
        self.step_count = 0

        self.schedule = RandomActivation(self)
        self.grid = SingleGrid(width, height, torus = False)

        self.floor = np.zeros((width, height))
        x = self.random.randrange(self.grid.width)
        y = self.random.randrange(self.grid.height)
        self.floor[x][y] = 10

        for i in range(self.num_explorers):
            agent = ExplorerAgent(i, self)
            self.schedule.add(agent)
            unplaced = True
            while unplaced:
                x = self.random.randrange(self.grid.width)
                y = self.random.randrange(self.grid.height)
                if self.floor[x][y] == 0:
                    if self.grid.is_cell_empty((x, y)):
                        unplaced = False
            self.grid.place_agent(agent, (x,y))

        for i in range(self.num_collectors):
            agent = CollectorAgent(i+self.num_explorers, self)
            self.schedule.add(agent)
            unplaced = True
            while unplaced:
                x = self.random.randrange(self.grid.width)
                y = self.random.randrange(self.grid.height)
                if self.floor[x][y] == 0:
                    if self.grid.is_cell_empty((x, y)):
                        unplaced = False
            self.grid.place_agent(agent, (x,y))
            
        self.datacollector = DataCollector(
            agent_reporters={"hasFood": "hasFood", "hasStorage": "hasStorage"},
            model_reporters={"Floor": self.get_floor, "AgentPositions": self.get_agent_positions}
        )


    def get_floor(self):
        return self.floor.copy()

    def place_food(self):
        if self.placed_food < self.count_food:
            num = self.random.randint(2,5)
            missing_food = self.count_food - self.placed_food
            val = min(num, missing_food)

            for i in range(val):
                unplaced = True
                while unplaced:
                    x = self.random.randrange(self.grid.width)
                    y = self.random.randrange(self.grid.height)
                    if self.floor[x][y] < 100:
                        if self.grid.is_cell_empty((x, y)):
                            unplaced = False
                self.floor[x][y] += 1
            self.placed_food += val

    def get_agent_positions(self):
        positions = np.zeros((self.grid.width, self.grid.height))

        for agent in self.schedule.agents:
            x, y = agent.pos
            positions[x][y] = agent.type

        return positions

    def step(self):
        self.step_count += 1
        if self.step_count % 5 == 0:
            self.place_food()
        self.schedule.step()
        self.datacollector.collect(self)

In [ ]:
WIDTH = 20
HEIGHT = 20
COUNT_FOOD = 47
NUM_EXPLORERS = 3
NUM_COLLECTORS = 2
STEPS = 1500

total_steps = 0

model = FoodModel(WIDTH, HEIGHT, NUM_EXPLORERS, NUM_COLLECTORS, COUNT_FOOD)

for i in range(STEPS):
    if model.collected_food < 47:
        model.step()
        total_steps += 1
    else:
        break

print(f"Steps: {total_steps}")
print(f"Food Collected: {model.collected_food}")

In [ ]:
all_grids = model.datacollector.get_model_vars_dataframe()

agentData = all_grids.get("AgentPositions")
floorData = all_grids.get("Floor")

In [ ]:
fig, axis = plt.subplots(figsize = (5,5))
plt.title("Food Storage")
axis.set_xticks([])
axis.set_yticks([])

all_data = all_grids.get("AgentPositions") * 10 + all_grids.get("Floor")
patch = axis.imshow(all_data[10], cmap = "binary")

def animate(frame):
    patch.set_data(all_data.iloc[frame])

anim = animation.FuncAnimation(fig, animate, frames = total_steps)

In [ ]:
anim

### **ESTADÍSTICA POR DIFERENTES VARIABLES**

In [ ]:
def get_model_statics(WIDTH, HEIGHT, NUM_EXPLORERS, NUM_COLLECTORS, COUNT_FOOD, STEPS):
    model = FoodModel(WIDTH, HEIGHT, NUM_EXPLORERS, NUM_COLLECTORS, COUNT_FOOD)
    for i in range(STEPS):
        if model.collected_food < COUNT_FOOD:
            model.step()
        else:
            break

    return [model.collected_food, model.step_count]

#### **NÚMERO DE PASOS POR DIFERENTES CANTIDADES DE AGENTES RECOLECTORES**

In [ ]:
WIDTH = 20
HEIGHT = 20
COUNT_FOOD = 47
NUM_EXPLORERS = 3
NUM_COLLECTORS = [1, 2, 3, 4, 5]
STEPS = 3500

steps = []

for i in NUM_COLLECTORS:
    steps.append(get_model_statics(WIDTH, HEIGHT, NUM_EXPLORERS, i, COUNT_FOOD, STEPS)[1])

plt.plot(NUM_COLLECTORS, steps)
plt.xlabel("Number of Collectors")
plt.ylabel("Steps")
plt.title("Number of Collectors vs Steps")
plt.show()

#### **NÚMERO DE PASOS POR DIFERENTES CANTIDADES DE AGENTES EXPLORADORES**

In [ ]:
WIDTH = 20
HEIGHT = 20
COUNT_FOOD = 47
NUM_EXPLORERS = [1, 2, 3, 4, 5]
NUM_COLLECTORS = 2
STEPS = 15000

steps = []

for i in NUM_EXPLORERS:
    steps.append(get_model_statics(WIDTH, HEIGHT, i, NUM_COLLECTORS, COUNT_FOOD, STEPS)[1])

plt.plot(NUM_EXPLORERS, steps)
plt.xlabel("Number of Explorers")
plt.ylabel("Steps")
plt.title("Number of Explorarers vs Steps")
plt.show()

In [ ]:
WIDTH = 20
HEIGHT = 20
COUNT_FOOD = 47
NUM_EXPLORERS = [1, 2, 3, 4]
NUM_COLLECTORS = [4, 3, 2, 1]
STEPS = 15000

steps = []

for i in range(len(NUM_EXPLORERS)):
    steps.append(get_model_statics(WIDTH, HEIGHT, NUM_EXPLORERS[i], NUM_COLLECTORS[i], COUNT_FOOD, STEPS)[1])

plt.plot([1,2,3,4], steps)
plt.xlabel("Number of Explorers")
plt.ylabel("Steps")
plt.title("Number of steps per different number of agents")
plt.show()